# 神经网络模型：

In [102]:
import torch

class Para:
    # tensor_board_log_dir = 'runs/exp0'
    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    # 模型设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification = 2 # 2, 3

    # 权重
    cross_weight = list()
    if classification == 3:
        cross_weight = [1.0, 1.0 ,1.0]
    elif classification == 2:
        cross_weight = [1.0, 1.0]
    elif classification == 5:
        cross_weight = [1.0, 1.0, 1.0, 1.0, 1.0]

    batch_size = 16
    lr = 1e-3
    drop = 0.5
    epochs = 100

    # 数据集设置
    month_in_sample = range(0, 1)
    # month_test = range(36, 48)

    percent_cv = 0.1 # 10% cross validation

    data_path = 'data/sk_space_1d_rate_20d_17-21_pre'


    seed = 2022
    torch.manual_seed(seed)

    info_str0 = data_path[5:7]+'_'+'c'+str(classification)+'_s'+str(percent_cv)
    info_str1 = '_b'+str(batch_size)+'_lr'+str(lr)+'_d'+str(drop)+'_e'+str(epochs)
    info_str = info_str0 + info_str1

    save_model_path = 'models/'+'model_'+info_str+'.pth'

para = Para()
print(para.info_str)
print(para.save_model_path)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.current_device())

sk_c2_s0.1_b16_lr0.001_d0.5_e100
models/model_sk_c2_s0.1_b16_lr0.001_d0.5_e100.pth
True
1
GeForce MX150
0


### 构建训练集

In [103]:
import numpy as np
import pandas as pd

data_in_sample = None
for i_month in para.month_in_sample:
    file_name = para.data_path + '/' + str(i_month) + '.csv'
    data_curr_month = pd.read_csv(file_name)

    data_curr_month = data_curr_month.dropna(axis=0)

    data_curr_month.insert(loc=0, column='return_bin', value=np.nan)

    data_curr_month.loc[data_curr_month['yield_rate']>0, 'return_bin'] = 0
    data_curr_month.loc[data_curr_month['yield_rate']<=0, 'return_bin'] = 1

    if i_month == para.month_in_sample[0]:
        data_in_sample = data_curr_month
    else:
        data_in_sample = pd.concat([data_in_sample, data_curr_month])
        # data_in_sample = data_in_sample.append(data_curr_month)

data_in_sample

,return_bin,order_book_id,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,1.0,600376.XSHG,MainBoard,RealEstate,0,2017-01-03,-0.235758,-1.411342,2.269983,-1.273296,...,-0.176158,1.939393,-0.343228,3.077250,2.097063,2.504830,0.386540,0.343156,-0.477163,-1.110336
1,1.0,600376.XSHG,MainBoard,RealEstate,1,2017-01-04,-0.212666,-1.419161,2.312394,-1.280733,...,0.563332,1.939393,-0.123865,3.058393,2.052508,2.496684,0.328454,0.339311,-0.623190,-1.059116
2,1.0,600376.XSHG,MainBoard,RealEstate,2,2017-01-05,-0.012847,-1.415701,2.293545,-1.277441,...,0.205077,1.267610,-0.129406,2.845400,2.009023,2.478314,0.140969,0.342943,-0.661480,-0.998378
3,0.0,600376.XSHG,MainBoard,RealEstate,3,2017-01-06,0.008439,-1.417434,2.302970,-1.279090,...,0.102341,1.939393,-0.032727,2.658063,1.981885,2.455540,-0.015838,0.342943,-0.541901,-0.971592
4,1.0,600376.XSHG,MainBoard,RealEstate,4,2017-01-09,-0.054613,-1.420023,2.317107,-1.281552,...,0.818119,1.939393,0.093412,2.005796,1.972691,2.419643,-0.342537,0.340627,-1.345192,-1.570440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,0.0,600376.XSHG,MainBoard,RealEstate,1192,2021-11-29,2.146251,1.209308,-1.466971,3.029146,...,0.019490,-0.747738,0.223932,-0.048353,-0.370257,-0.790084,-1.188693,-0.353637,-0.928837,-1.234122
1190,0.0,600376.XSHG,MainBoard,RealEstate,1193,2021-11-30,2.398559,1.232542,-1.478548,3.062748,...,-0.058182,-0.747738,0.067898,-0.101384,-0.371908,-0.790859,-1.197434,-0.333250,-0.439751,-0.827627
1191,0.0,600376.XSHG,MainBoard,RealEstate,1194,2021-12-01,2.238078,1.186366,-1.455393,2.995967,...,0.530404,-0.075955,0.357910,-0.112933,-0.373548,-0.786568,-1.193840,-0.319589,-0.044396,-0.533675
1192,0.0,600376.XSHG,MainBoard,RealEstate,1195,2021-12-02,2.176447,1.126572,-1.424521,2.909491,...,0.998287,-0.075955,0.710331,-0.095406,-0.369987,-0.776094,-1.137295,-0.315319,0.754954,-0.036107


In [104]:
X_in_sample = data_in_sample.loc[:, para.feature_column_start_name: para.feature_column_end_name]
y_in_sample = data_in_sample.loc[:, 'return_bin']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False) # True, random_state=para.seed)
# X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False)

X_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,-1.411342,2.269983,-1.273296,-1.025096,-0.866096,-0.652229,-0.176158,1.939393,-0.343228,3.077250,2.097063,2.504830,0.386540,0.343156,-0.477163,-1.110336
1,-1.419161,2.312394,-1.280733,-0.923666,-0.893492,-0.290177,0.563332,1.939393,-0.123865,3.058393,2.052508,2.496684,0.328454,0.339311,-0.623190,-1.059116
2,-1.415701,2.293545,-1.277441,-0.852875,-0.900033,-0.070599,0.205077,1.267610,-0.129406,2.845400,2.009023,2.478314,0.140969,0.342943,-0.661480,-0.998378
3,-1.417434,2.302970,-1.279090,-0.774502,-0.888245,0.122402,0.102341,1.939393,-0.032727,2.658063,1.981885,2.455540,-0.015838,0.342943,-0.541901,-0.971592
4,-1.420023,2.317107,-1.281552,-0.685683,-0.859525,0.300690,0.818119,1.939393,0.093412,2.005796,1.972691,2.419643,-0.342537,0.340627,-1.345192,-1.570440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,0.865492,-1.035077,1.289191,-0.005168,-0.109375,0.271298,1.026958,-0.075955,0.431593,-0.109275,-0.466151,-0.773933,-0.969268,-0.918862,-0.714858,-0.785255
1070,0.848154,-1.023156,1.270623,0.035801,-0.079734,0.310384,1.090179,0.595827,0.504560,-0.328337,-0.474823,-0.773665,-0.990972,-0.924201,-0.401836,-0.627900
1071,0.853913,-1.027130,1.276791,0.064486,-0.049791,0.313559,1.090179,-0.075955,0.411834,-0.432338,-0.500946,-0.774325,-1.052036,-0.927824,-0.324829,-0.482743
1072,0.871313,-1.039051,1.295425,0.072421,-0.024114,0.268650,0.497477,-0.075955,0.240095,-0.522253,-0.522041,-0.776736,-1.180918,-0.933460,-0.261731,-0.653516


In [105]:
y_train

0       1.0
1       1.0
2       1.0
3       0.0
4       1.0
       ... 
1069    1.0
1070    1.0
1071    1.0
1072    0.0
1073    1.0
Name: return_bin, Length: 1074, dtype: float64

In [106]:
data_train = pd.concat([X_train, y_train], axis=1)
data_cv = pd.concat([X_cv, y_cv], axis=1)
data_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR,return_bin
0,-1.411342,2.269983,-1.273296,-1.025096,-0.866096,-0.652229,-0.176158,1.939393,-0.343228,3.077250,2.097063,2.504830,0.386540,0.343156,-0.477163,-1.110336,1.0
1,-1.419161,2.312394,-1.280733,-0.923666,-0.893492,-0.290177,0.563332,1.939393,-0.123865,3.058393,2.052508,2.496684,0.328454,0.339311,-0.623190,-1.059116,1.0
2,-1.415701,2.293545,-1.277441,-0.852875,-0.900033,-0.070599,0.205077,1.267610,-0.129406,2.845400,2.009023,2.478314,0.140969,0.342943,-0.661480,-0.998378,1.0
3,-1.417434,2.302970,-1.279090,-0.774502,-0.888245,0.122402,0.102341,1.939393,-0.032727,2.658063,1.981885,2.455540,-0.015838,0.342943,-0.541901,-0.971592,0.0
4,-1.420023,2.317107,-1.281552,-0.685683,-0.859525,0.300690,0.818119,1.939393,0.093412,2.005796,1.972691,2.419643,-0.342537,0.340627,-1.345192,-1.570440,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,0.865492,-1.035077,1.289191,-0.005168,-0.109375,0.271298,1.026958,-0.075955,0.431593,-0.109275,-0.466151,-0.773933,-0.969268,-0.918862,-0.714858,-0.785255,1.0
1070,0.848154,-1.023156,1.270623,0.035801,-0.079734,0.310384,1.090179,0.595827,0.504560,-0.328337,-0.474823,-0.773665,-0.990972,-0.924201,-0.401836,-0.627900,1.0
1071,0.853913,-1.027130,1.276791,0.064486,-0.049791,0.313559,1.090179,-0.075955,0.411834,-0.432338,-0.500946,-0.774325,-1.052036,-0.927824,-0.324829,-0.482743,1.0
1072,0.871313,-1.039051,1.295425,0.072421,-0.024114,0.268650,0.497477,-0.075955,0.240095,-0.522253,-0.522041,-0.776736,-1.180918,-0.933460,-0.261731,-0.653516,0.0


In [107]:
from torch.utils.data import TensorDataset
import torch

X_train_ndarray = X_train.values
y_train_ndarray = y_train.values

train_dataset = TensorDataset(torch.from_numpy(X_train_ndarray).type(torch.FloatTensor), torch.from_numpy(y_train_ndarray).type(torch.LongTensor))

# for X_train_temp, y_train_temp in train_dataset:
#     print(X_train_temp, y_train_temp)
#     print(X_train_temp.dtype, y_train_temp.dtype)

In [108]:
X_cv_ndarray = X_cv.values
y_cv_ndarray = y_cv.values

cv_dataset = TensorDataset(torch.from_numpy(X_cv_ndarray).type(torch.FloatTensor), torch.from_numpy(y_cv.values).type(torch.LongTensor))

# for X_cv_temp, y_cv_temp in cv_dataset:
#     print(X_cv_temp, y_cv_temp)
#     print(X_cv_temp.dtype, y_cv_temp.dtype)

In [109]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

cv_dataloader = DataLoader(
    dataset=cv_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

### 构建测试集

In [110]:
# data_test = None
# for i_month in para.month_test:
#
#     file_name = para.data_path + '/' + str(i_month) + '.csv'
#     data_curr_month = pd.read_csv(file_name)
#
#     data_curr_month = data_curr_month.dropna(axis=0)
#
#     data_curr_month = label_data(data=data_curr_month, percent_select=para.percent_select)
#
#     if i_month == para.month_test[0]:
#         data_test = data_curr_month
#     else:
#         data_test = pd.concat([data_test, data_curr_month])
#         # data_test = data_test.append(data_curr_month)
#
# X_test = data_test.loc[:, para.feature_column_start_name: para.feature_column_end_name]
# y_test = data_test.loc[:, 'return_bin']

In [111]:
# from torch.utils.data import TensorDataset
# import torch
#
#
# X_test_ndarray = X_test.values
# y_test_ndarray = y_test.values
#
# test_dataset = TensorDataset(torch.from_numpy(X_test_ndarray).type(torch.FloatTensor), torch.from_numpy(y_test_ndarray).type(torch.LongTensor))

In [112]:
# from torch.utils.data import DataLoader
#
#
# test_dataloader = DataLoader(
#     dataset=test_dataset,
#     batch_size=para.batch_size,
#     shuffle=True,
# )

### 构建神经网络

In [113]:
from my_utils.model_class import MLP

model = MLP(in_nums=len(X_train.columns), out_nums=para.classification, drop_p=para.drop)
# to device
model = model.to(device=para.device)
print(model)

MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
)


### 训练与测试

In [114]:
import torchmetrics


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # initialize metric
    train_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    for batch, (X, y) in enumerate(dataloader):

        # to device
        X = X.to(device=para.device)
        y = y.to(device=para.device)

        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # metric on current batch
        train_precision(pred.argmax(1), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = train_precision.compute()
    print("Precision of every train dataset class: ", total_precision)
    print()

    return correct, train_loss, total_precision


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # initialize metric
    test_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    with torch.no_grad():
        for X, y in dataloader:

            # to device
            X = X.to(device=para.device)
            y = y.to(device=para.device)

            # compute prediction and loss
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # metric on current batch
            test_precision(pred.argmax(1), y)


    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = test_precision.compute()
    print("Precision of every test dataset class: ", total_precision)
    print()

    return correct, test_loss, total_precision

In [115]:
def select_df_to_dataloader(df: pd.DataFrame, select: int) -> DataLoader:

    df = df[df['return_bin'] == select]

    df_dataset = TensorDataset(
        torch.from_numpy(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).type(torch.FloatTensor),
        torch.from_numpy(df.loc[:, 'return_bin'].values).type(torch.LongTensor))

    df_dataloader = DataLoader(
        dataset=df_dataset,
        batch_size=para.batch_size,
        shuffle=True,
    )

    return df_dataloader

In [116]:
# temp2_dataloader = select_df_to_dataloader(df=data_cv, select=2)
temp1_dataloader = select_df_to_dataloader(df=data_cv, select=1)
temp0_dataloader = select_df_to_dataloader(df=data_cv, select=0)

In [117]:
from torch import nn
import time
from torch.utils.tensorboard import SummaryWriter

# 计时
time_start = time.time()

# writer = SummaryWriter(para.tensor_board_log_dir)
writer = SummaryWriter()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# to device
loss_fn = loss_fn.to(device=para.device)

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=para.lr)

epochs = para.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    accuracy_train, loss_train, precision_train = train_loop(train_dataloader, model, loss_fn, optimizer)
    model.eval()
    accuracy_cv, loss_cv, precision_cv = test_loop(cv_dataloader, model, loss_fn)

    # accuracy2 = test_loop(temp2_dataloader, model, loss_fn)
    # print('#')
    # accuracy1 = test_loop(temp1_dataloader, model, loss_fn)
    # accuracy0 = test_loop(temp0_dataloader, model, loss_fn)

    # 写入 tensorboard
    if para.classification == 2:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1]},
                           global_step=t)

    elif para.classification == 3:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2]},
                           global_step=t)

    elif para.classification == 5:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2],
                               'precision3': precision_cv[3],
                               'precision4': precision_cv[4]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2],
                               'precision3': precision_train[3],
                               'precision4': precision_train[4]},
                           global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/cv',
                       tag_scalar_dict={
                           'loss': loss_cv},
                       global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/train',
                       tag_scalar_dict={
                           'loss': loss_train},
                       global_step=t)
    writer.flush()

    time_end = time.time()
    print('Time cost = %fs' % (time_end - time_start))
    print()

writer.close()

print("Done!")

Epoch 1
-------------------------------
Train Error: 
    Accuracy: 52.9%, Avg loss: 0.691103 

Precision of every train dataset class:  tensor([0.5143, 0.5370], device='cuda:0')

Test Error: 
    Accuracy: 56.7%, Avg loss: 0.683918 

Precision of every test dataset class:  tensor([0.5395, 0.6136], device='cuda:0')

Time cost = 1.108734s

Epoch 2
-------------------------------
Train Error: 
    Accuracy: 55.4%, Avg loss: 0.684770 

Precision of every train dataset class:  tensor([0.5549, 0.5536], device='cuda:0')

Test Error: 
    Accuracy: 70.0%, Avg loss: 0.681456 

Precision of every test dataset class:  tensor([0.7391, 0.6757], device='cuda:0')

Time cost = 2.079138s

Epoch 3
-------------------------------
Train Error: 
    Accuracy: 54.7%, Avg loss: 0.686226 

Precision of every train dataset class:  tensor([0.5452, 0.5472], device='cuda:0')

Test Error: 
    Accuracy: 73.3%, Avg loss: 0.676872 

Precision of every test dataset class:  tensor([0.8611, 0.6786], device='cuda:0')



## 保存模型

In [118]:
torch.save(model.state_dict(), para.save_model_path)

print('Finish save model!')

Finish save model!


## captum

In [119]:
# # captum
# from captum.attr import IntegratedGradients
#
# ig = IntegratedGradients(model)

In [120]:
# temp = cv_dataloader.dataset.tensors[0]
# temp.requires_grad_()
# attr, delta = ig.attribute(temp,target=1, return_convergence_delta=True)
# attr = attr.detach().numpy()

In [121]:
# # Helper method to print importances and visualize distribution
# def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
#     print(title)
#     for i in range(len(feature_names)):
#         print(feature_names[i], ": ", '%.3f'%(importances[i]))
#     y_pos = (np.arange(len(feature_names)))
#     if plot:
#         plt.figure(figsize=(20,6))
#         plt.barh(y_pos, importances, align='center')
#         plt.yticks(y_pos, feature_names)
#         plt.ylabel(axis_title)
#         plt.grid(axis='y')
#         plt.title(title)
# visualize_importances(feature_names=X_cv.columns.values.tolist(), importances=np.mean(attr, axis=0))

In [122]:
# X_cv.columns.values.tolist()

In [123]:
# loss = nn.CrossEntropyLoss()
# input = torch.Tensor(
#     [[-0.0441,  0.0773],
#     [-0.0781, -0.1772],
#     [-0.1319, -0.0432],
#     [-0.0714, -0.1261],
#     [-0.0806, -0.1370],
#     [-0.1730, -0.1472],
#     [-0.0350, -0.0507],
#     [-0.1149, -0.2248]])
# # input = input.reshape(-1,4)
# target = torch.Tensor([0, 1, 1, 0, 0, 0, 0, 0]).type(torch.LongTensor)
# print(input.dtype)
# print(target.dtype)
# output = loss(input, target)
# print(input, target, output)

In [124]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.Tensor([1,4,1]).type(torch.LongTensor)
# output = loss(input, target)
# print(input,target,output)

In [125]:
# loss = nn.BCEWithLogitsLoss()
# input = torch.Tensor([0.5, 0.4, 0.3])
# target = torch.Tensor([0])
# output = loss(input, target)
# print(input, target, output)